In [8]:
import cv2
import numpy as np
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [9]:
# Transfertraining facefinder
# eye_cascade = cv2.CascadeClassifier('./haarcascade_eye.xml')
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

In [10]:
# init classifier
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# This layer creates a convolution kernel that is convolved with the layer input to produce a tensor of outputs: 
# 32 Random filters each of size 3x3 -- input shape of 64x64 RGB
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Max Pooling is typically added to CNN after individual convolutional layers
# It basically scales down the number of pixels in the output of the convo. layers by some factor (in the case by factor of 2[half])
classifier.add(Flatten())
# this essentially turns our input shape into a flattened vector of shape 64*64*3 = 12288 
classifier.add(Dense(units = 128, activation = 'relu'))
# First activatio layer (hidden) applies ReLU and transforms output dimensionality of 128 in the resulting output space
# ReLU (Rectified Linear Unit) function when applied it will convert all negative numbers to 0 or positive.
classifier.add(Dense(units = 1, activation = 'sigmoid'))
# Second hidden layer 
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.load_weights("classifier.h5")

In [11]:
# init the labels dict in a weird way since i dont know a shortcut(this piece comes from model training)
# ImageDataGenerator - Generates batches of tensor image data with real-time data augmentation. The data will be looped over (in batches).
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

training_set = train_datagen.flow_from_directory('./pics/',
target_size = (64, 64),
batch_size = 32,
class_mode = 'binary')

labels = dict((value, key) for key, value in training_set.class_indices.items())

Found 87 images belonging to 1 classes.


In [12]:
cap = cv2.VideoCapture(0) # number specifies the camera which to use 0 = my only webcam
while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w,y+h), (255, 0, 0), 2) # (image, start bottom L, end top R, color, line-width)
        roi_gray = gray[y:y+h,x:x+w]
        roi_color = img[y:y+h,x:x+w]
        
        pred_image = roi_color
        pred_image = np.stack((np.dot(roi_color[...,:3], [1/255, 1/255, 1/255]),)*3, -1)
        pred_image = cv2.resize(pred_image, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
        # test_image = pred_image
        pred_image = np.expand_dims(pred_image, axis = 0)
        result = classifier.predict(pred_image)
        cv2.putText(img,labels.get(round(result[0][0])),(x+10, y+30), 0, 1, (0, 0, 255),2)
        
        # eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        # for (ex, ey, ew, eh) in eyes:
        #     cv2.rectangle(roi_color, (ex, ey), (ex+ew,ey+eh), (0, 255, 0), 2)
    cv2.imshow('img',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:   
        break
cap.release()
cv2.destroyAllWindows()